# This Notebook is used to scrape the Wikipedia page with Toronto neighbourhoods.

# Part I. Web Sraping

#### Installation of the required packages.

In [162]:
!pip install opencage
#!conda install beautifulsoup4
#!conda install -c conda-forge folium=0.5.0 --yes

#### Import of the required modules.

In [163]:
import lxml
import folium
import requests
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors

from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
from opencage.geocoder import OpenCageGeocode

#### Getting the Neighbourhoods table from the Wikipedia page.

In [164]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
soup = BeautifulSoup(requests.get(url).content,features = 'lxml')
table = soup.find_all('table',class_ = 'wikitable sortable')[0]
df = pd.read_html(str(table))[0]

#### Transforming the data into the required form.

In [165]:
df = df.loc[(df['Borough'] != 'Not assigned')]
mask = (df['Neighbourhood'] == 'Not assigned')
df['Neighbourhood'][mask] = df['Borough'][mask]
df = df.sort_values(['Postcode','Borough','Neighbourhood'])
df = df.groupby(['Postcode','Borough'],as_index = False).agg({'Neighbourhood':', '.join})
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Printing the number of rows and columns of the resulting dataframe.

In [166]:
print("The number of rows and columns of the resulting dataframe:",df.shape)

The number of rows and columns of the resulting dataframe: (103, 3)


# Part II. Getting the coordinates for each neighbourhood.

#### Getting the geocodes for each Postcode.

As neither "geopy.geocoders" nor "geocoder.google" did not work for me; I started to look for something else.
After some googling, I've found "opencage.geocoder".  
This worked well, excluding three cases, where opencage did not find the postcodes: M3A, M7A, M7R. 
I've updated their latitudes and longitudes manually.

In [167]:
# The code was removed by Watson Studio for sharing.

In [168]:
geocoder = OpenCageGeocode(key)
latitude_list = []
longitude_list = []
response = []

for i in df.index:
    address = df['Postcode'][i] + ', Toronto, Ontario'   
    results = geocoder.geocode(address) 
    for j in range(0,len(results)):
        if df['Postcode'][i] in results[j]['formatted']:
            latitude = results[j]['geometry']['lat']
            longitude = results[j]['geometry']['lng']
            break
        else:
            latitude = 0
            longitude = 0
    response.append(results[j]['formatted'])            
    latitude_list.append(latitude)
    longitude_list.append(longitude)
    #print('The geograpical coordinate are {}, {}.'.format(latitude, longitude))

df['Latitude'] = latitude_list
df['Longitude'] = longitude_list
df['response'] = response

df.loc[(df['Latitude'] == 0)]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,response
25,M3A,North York,Parkwoods,0.0,0.0,"Toronto, ON M6K 1X9, Canada"
85,M7A,Downtown Toronto,Queen's Park,0.0,0.0,"Toronto, ON M6K 1X9, Canada"
86,M7R,Mississauga,Canada Post Gateway Processing Centre,0.0,0.0,"Toronto, ON M6K 1X9, Canada"


As mentioned above, there are three cases where opencage cannot find the requested postcodes.
That is getting obvious after the opencage response check.  
Therefore, in the code below, the geocodes have been updated manually.

In [169]:
df.loc[df['Postcode'] == 'M3A','Latitude'],df.loc[df['Postcode'] == 'M3A','Longitude']  = 43.7533,-79.3297
df.loc[df['Postcode'] == 'M7A','Latitude'],df.loc[df['Postcode'] == 'M7A','Longitude']  = 43.6623,-79.3895
df.loc[df['Postcode'] == 'M7R','Latitude'],df.loc[df['Postcode'] == 'M7R','Longitude']  = 43.6370,-79.6158
df.drop('response',axis = 1, inplace = True)
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.811300,-79.193000
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.787800,-79.156400
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.767800,-79.186600
3,M1G,Scarborough,Woburn,43.765717,-79.221898
4,M1H,Scarborough,Cedarbrae,43.768600,-79.238900


#### Create a map with Toronto neighborhoods.

In [170]:
toronto_coord = geocoder.geocode('Toronto, Ontario')
toronto_lat = toronto_coord[0]['geometry']['lat']
toronto_lng = toronto_coord[0]['geometry']['lng']

map_toronto = folium.Map(location = [toronto_lat,toronto_lng],zoom_start = 10)

for lat,lng,borough,neighbourhood in zip(df['Latitude'],df['Longitude'],df['Borough'],df['Neighbourhood']):
    label = '{}: {}'.format(neighbourhood,borough)
    label = folium.Popup(label,parse_html = True)
    folium.CircleMarker([lat,lng],radius = 5,popup = label,color = 'blue',fill = True,
                        fill_color = '#3186cc',fill_opacity = 0.7,parse_html = False).add_to(map_toronto)
map_toronto

## Part III. Exploring the neighbourhoods in Toronto

In [32]:
# The code was removed by Watson Studio for sharing.

#### Definition of the function "getNearbyVenues". This function is dedicated to get the venue list (max 100) for each neighbourhood in a 500m radius. 

In [176]:
def getNearbyVenues(neighbourhoods,latitudes,longitudes,radius = 500,limit = 3):    
    venues_list = []
    for neighbourhood,lat,lng in zip(neighbourhoods,latitudes,longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,limit)
        results = requests.get(url).json()["response"]['groups'][0]['items']        
        venues_list.append([(neighbourhood,lat,lng,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name'])
                            for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood','Neighbourhood Latitude','Neighbourhood Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category']
    return(nearby_venues)

#### Getting the venues list for Toronto neighbourhoods.

In [177]:
toronto_venues = getNearbyVenues(neighbourhoods = df['Neighbourhood'],latitudes = df['Latitude'],longitudes = df['Longitude'],limit = 100)

Let's check how many venues were returned for each neighborhood.

In [179]:
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',1000)
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",13,13,13,13,13,13
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Downsview North, Wilson Heights",23,23,23,23,23,23
"Bathurst Quay, CN Tower, Harbourfront West, Island airport, King and Spadina, Railway Lands, South Niagara",100,100,100,100,100,100
Bayview Village,3,3,3,3,3,3
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25


In [203]:
list(set(df['Neighbourhood'].values) - set(toronto_venues['Neighbourhood'].values))

['Highland Creek, Port Union, Rouge Hill', 'Upper Rouge']

In [213]:
len(df['Neighbourhood'])

103

In [214]:
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.811300,-79.193000
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.787800,-79.156400
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.767800,-79.186600
3,M1G,Scarborough,Woburn,43.765717,-79.221898
4,M1H,Scarborough,Cedarbrae,43.768600,-79.238900
5,M1J,Scarborough,Scarborough Village,43.746400,-79.232300
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.729800,-79.263900
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.712200,-79.284300
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.724700,-79.231200
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.695200,-79.264600


In [207]:
toronto_venues.groupby('Neighbourhood').count().shape

(100, 6)

Let's find out how many unique venue categories we have received from the Foursquare API.

In [180]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 275 uniques categories.


#### One hot encoding. Let's add venue categories as the columns.

In [187]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']],prefix = '',prefix_sep = '')
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 
toronto_onehot = toronto_onehot[[toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])]
toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stationery Store,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio
0,"Malvern, Rouge"

In [188]:
print('The new dataframe size: {}.'.format(toronto_onehot.shape))

The new dataframe size: (2449, 276).


#### Now, let's group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category.

In [189]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Soccer Field,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stationery Store,Steakhouse,Street Art,Strip Club,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio
0,"Adelaide, King,

In [190]:
print('The new dataframe size: {}.'.format(toronto_grouped.shape))

The new dataframe size: (100, 276).


#### Let's print each neighborhood along with the top 5 most common type of venues.

In [191]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print('-----' + hood + '-----')
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq',ascending = False).reset_index(drop = True).head(num_top_venues))
    print('\n')


-----Adelaide, King, Richmond-----
              venue  freq
0              Café  0.06
1       Coffee Shop  0.06
2             Hotel  0.04
3        Steakhouse  0.04
4  Asian Restaurant  0.03


-----Agincourt-----
                       venue  freq
0             Breakfast Spot   0.2
1            Badminton Court   0.2
2        Shanghai Restaurant   0.2
3  Latin American Restaurant   0.2
4               Skating Rink   0.2


-----Agincourt North, L'Amoreaux East, Milliken, Steeles East-----
                        venue  freq
0                    Pharmacy   0.5
1            Sushi Restaurant   0.5
2           Accessories Store   0.0
3  Modern European Restaurant   0.0
4                  Nail Salon   0.0


-----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown-----
            venue  freq
0   Grocery Store  0.15
1     Pizza Place  0.08
2    Liquor Store  0.08
3  Sandwich Place  0.08
4      Beer Store  0.08


-----Alderwood, Long Bra

In [64]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [138]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Hotel,Restaurant,Bakery,Gym,Asian Restaurant,Breakfast Spot,Burger Joint
1,Agincourt,Breakfast Spot,Latin American Restaurant,Badminton Court,Shanghai Restaurant,Skating Rink,Event Space,Ethiopian Restaurant,Electronics Store,Falafel Restaurant,Dim Sum Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Sushi Restaurant,Yoga Studio,Dumpling Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Auto Garage,Hardware Store,Coffee Shop,Caribbean Restaurant,Sandwich Place,Fast Food Restaurant,Liquor Store,Beer Store
4,"Alderwood, Long Branch",Pizza Place,Sandwich Place,Coffee Shop,Convenience Store,Pub,Pharmacy,Gym,Greek Restaurant,Curling Ice,Dumpling Restaurant


In [139]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 2, 0, 0], dtype=int32)

In [140]:
#neighbourhoods_venues_sorted.loc[neighbourhoods_venues_sorted['Cluster Labels'] == 2]

In [141]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.811300,-79.193000,3.0,Home Service,Yoga Studio,Dumpling Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.787800,-79.156400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.767800,-79.186600,0.0,Pizza Place,Fast Food Restaurant,Grocery Store,Pharmacy,Coffee Shop,Discount Store,Intersection,Food & Drink Shop,Beer Store,Supermarket
3,M1G,Scarborough,Woburn,43.765717,-79.221898,0.0,Taco Place,Indian Restaurant,American Restaurant,Park,Coffee Shop,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant
4,M1H,Scarborough,Cedarbrae,43.768600,-79.238900,0.0,Lounge,Trail,Construction & Landscaping,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant


In [142]:
toronto_merged.dropna(inplace = True)
toronto_merged.astype({'Cluster Labels': 'int64'},inplace = True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.811300,-79.193000,3,Home Service,Yoga Studio,Dumpling Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.767800,-79.186600,0,Pizza Place,Fast Food Restaurant,Grocery Store,Pharmacy,Coffee Shop,Discount Store,Intersection,Food & Drink Shop,Beer Store,Supermarket
3,M1G,Scarborough,Woburn,43.765717,-79.221898,0,Taco Place,Indian Restaurant,American Restaurant,Park,Coffee Shop,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant
4,M1H,Scarborough,Cedarbrae,43.768600,-79.238900,0,Lounge,Trail,Construction & Landscaping,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
5,M1J,Scarborough,Scarborough Village,43.746400,-79.232300,0,Spa,Grocery Store,Yoga Studio,Dumpling Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


In [143]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1

In [145]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,0,Pizza Place,Fast Food Restaurant,Grocery Store,Pharmacy,Coffee Shop,Discount Store,Intersection,Food & Drink Shop,Beer Store,Supermarket
3,Scarborough,0,Taco Place,Indian Restaurant,American Restaurant,Park,Coffee Shop,Yoga Studio,Diner,Discount Store,Dog Run,Doner Restaurant
4,Scarborough,0,Lounge,Trail,Construction & Landscaping,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
5,Scarborough,0,Spa,Grocery Store,Yoga Studio,Dumpling Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
6,Scarborough,0,Coffee Shop,Pharmacy,Metro Station,Bus Line,Light Rail Station,Sandwich Place,Discount Store,Department Store,Intersection,Chinese Restaurant
7,Scarborough,0,Bakery,Intersection,Bus Line,Bus Station,Soccer Field,Coffee Shop,Park,Donut Shop,Diner,Falafel Restaurant
8,Scarborough,0,Ice Cream Shop,Liquor Store,Pizza Place,Coffee Shop,Sandwich Place,Bistro,Bank,Pharmacy,Gluten-free Restaurant,Event Space
9,Scarborough,0,Skating Rink,General Entertainment,Café,College Stadium,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
10,Scarborough,0,Construction & Landscaping,Bakery,Asian Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
12,Scarborough,0,Breakfast Spot,Latin American Restaurant,Badminton Court,Shanghai Restaurant,Skating Rink,Event Space,Ethiopian Restaurant,Electronics Store,Falafel Restaurant,Dim Sum Restaurant


#### Cluster 2

In [148]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,1,Auto Garage,Yoga Studio,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


#### Cluster 3

In [149]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,North York,2,Park,Residential Building (Apartment / Condo),Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
18,North York,2,Park,Pharmacy,Pizza Place,Wine Shop,Coffee Shop,Thai Restaurant,Convenience Store,Middle Eastern Restaurant,Dog Run,Department Store
19,North York,2,Gas Station,Trail,Park,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
21,North York,2,Playground,Gym,Yoga Studio,Dumpling Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
23,North York,2,Convenience Store,Park,Bank,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
24,North York,2,Baby Store,Park,Coffee Shop,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
25,North York,2,Food & Drink Shop,Park,Yoga Studio,Dumpling Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
26,North York,2,Basketball Court,Construction & Landscaping,Pool,Park,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
40,East York,2,Convenience Store,Park,Coffee Shop,Greek Restaurant,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
44,Central Toronto,2,Photography Studio,Park,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant


#### Cluster 4

In [150]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,3,Home Service,Yoga Studio,Dumpling Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant
96,North York,3,Home Service,Yoga Studio,Dumpling Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant


#### Cluster 5

In [151]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,North York,4,Baseball Field,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Dessert Shop
95,Etobicoke,4,Baseball Field,Yoga Studio,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Dessert Shop
